In [ ]:
import os
import healpy as hp
import numpy as np
import parfiles.Noise.parfile as par
import parfiles.Noise.parfile_delensed as par_del

output_TT = []
output_EE = []
output_TE = []

for i in range(60, 300):
    tlm = par.ivfs.get_sim_tmliklm(i)
    # elm = par.ivfs.get_sim_emliklm(i)
    tlm_del = par_del.ivfs.get_sim_tmliklm(i)
    # elm_del = par_del.ivfs.get_sim_emliklm(i)

    dcl = hp.alm2cl(tlm_del) - hp.alm2cl(tlm)
    output_TT.append(dcl)

    # dcl = hp.alm2cl(elm_del) - hp.alm2cl(elm)
    # output_EE.append(dcl)

    # dcl = hp.alm2cl(tlm_del, elm_del) - hp.alm2cl(tlm, elm)
    # output_TE.append(dcl)

np.save("TT_carron17", output_TT)
# np.save("EE_carron17", output_EE)
# np.save("TE_carron17", output_TE)

Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_lensed/ivfs/sim_0060_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_delensed/ivfs/sim_0060_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_lensed/ivfs/sim_0061_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_delensed/ivfs/sim_0061_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_lensed/ivfs/sim_0062_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_delensed/ivfs/sim_0062_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_lensed/ivfs/sim_0063_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_delensed/ivfs/sim_0063_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_lensed/ivfs/sim_0064_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_delensed/ivfs/sim_0064_tlm.fits
Filt library_sepTP: Loading /mnt/d/THESIS/PLENS/noisy_ivfs_lensed/ivfs/sim_0065_tlm.fits
Filt librar

In [ ]:
import os
import numpy as np
import parfiles.Noise.parfile as par
import matplotlib.pyplot as plt

#--- load and average your simulation ΔCℓ arrays
dcl_sim = {
    'tt': np.average(np.load("TT_carron17.npy"), axis=0),
    'te': np.average(np.load("TE_carron17.npy"), axis=0),
    'ee': np.average(np.load("EE_carron17.npy"), axis=0),
}

#--- theory unlensed minus lensed
cl_len = {s: par.cl_len[s][:2049]   for s in ['tt','te','ee']}
cl_unl = {s: par.cl_unl[s][:2049]   for s in ['tt','te','ee']}
dcl_th  = {s: cl_unl[s] - cl_len[s] for s in ['tt','te','ee']}

#--- ℓ array (used for binning)
ell = np.arange(len(cl_len['tt']))

#--- zero out low‐ℓ
for s in ['tt','te','ee']:
    dcl_sim[s][:100] = 0
    dcl_th [s][:100] = 0

#--- bin setup (same as before)
bins        = np.arange(100, 2001, 15)
bin_centers = 0.5*(bins[:-1] + bins[1:])

def bin_cl(ell, cl, bins):
    nb = len(bins)-1
    clb = np.zeros(nb)
    for i in range(nb):
        sel = (ell>=bins[i]) & (ell<bins[i+1])
        w   = (2*ell[sel]+1)
        clb[i] = np.sum(w*cl[sel]) / np.sum(w)
    return clb

#--- bin them all
dcl_sim_b = {s: bin_cl(ell, dcl_sim[s], bins) for s in ['tt','te','ee']}
dcl_th_b  = {s: bin_cl(ell, dcl_th [s], bins) for s in ['tt','te','ee']}

#--- define Dℓ scaling factors: TT ~ ℓ^2, TE ~ ℓ^1, EE ~ ℓ^0
scale = {
    'tt': bin_centers**2/(2*np.pi),
    'te': bin_centers  /(2*np.pi),
    'ee': np.ones_like(bin_centers)/(2*np.pi),
}

#--- plotting
titles = {'tt':'TT', 'te':'TE', 'ee':'EE'}
fig, axes = plt.subplots(3, 1, figsize=(6,5), sharex=True)

for ax, spec in zip(axes, ['tt','te','ee']):
    pwr = 2 if spec=='tt' else (1 if spec=='te' else 0)
    ylabel = fr"{titles[spec]}:  $\ell^{{{pwr}}}\, \Delta C_\ell/2\pi$ [μK^2]"

    ax.plot(bin_centers, dcl_sim_b[spec] * scale[spec], label=r'$\langle\Delta C_\ell\rangle_{\rm sims}$', color='C0')
    ax.plot(bin_centers, dcl_th_b [spec] * scale[spec], label=r'$\Delta C_\ell^{\rm theory}$', color='C1')
    ax.axhline(0, color='gray', linestyle='--')
    ax.set_ylabel(ylabel, size=8)
    if spec=='tt':
        ax.set_title("Change in TT/TE/EE Power Spectra Due to Delensing: Theory vs Simulations", size=10)
        ax.legend(loc='upper right')

axes[-1].set_xlabel(r'$\ell$')
plt.xlim(100, 1500)
plt.tight_layout()
plt.savefig("fig_carron17_check.png")
plt.close()

